In [54]:
import pandas as pd
from collections import Counter
from nltk.util import ngrams
import re
import string

In [59]:
def clean_text(text):
    text = str(text).strip() 
    text = text.replace("\n", " ") 
    text = text.strip('"')  
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  
    text = re.sub(' +', ' ', text)  
    return text.lower() 

def tokenize_text(text, n=6):
    after_first_colon = text.split(":", 1)[-1] 
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))  # map punctuation to space
    after_first_colon = after_first_colon.translate(translator) 
    words = after_first_colon.split()
    return [tuple(words[i:i+n]) for i in range(len(words)-n+1)]

def count_frequencies(token_list):
    return Counter(token_list)

In [60]:
df = pd.read_csv('enriched_BST_data_v2.csv')

In [64]:
df['cyto_report'] = df['text_data'].apply(clean_text)
df['concurrent_report'] = df['concurrent_report'].apply(clean_text)

In [65]:
df.to_csv('enriched_BST_data_v2.csv', index=False)

In [35]:
n = 6
df['tokens'] = df['clean_conc_report'].apply(lambda x: tokenize_text(x, n))

In [36]:
all_tokens = [token for sublist in df['tokens'].tolist() for token in sublist]
token_frequencies = count_frequencies(all_tokens)

In [37]:
sorted_token_frequencies = sorted(token_frequencies.items(), key=lambda x: x[1], reverse=True)

In [38]:
df_tokens = pd.DataFrame(sorted_token_frequencies, columns=['Token', 'Frequency'])
df_tokens.to_csv('token_freq.csv', index=False)